In [1]:
import numpy
from pybit.unified_trading import HTTP
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from models.TS.GRUPipeline import GRUNetwork
import time
from torch import optim
from torch import nn
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

In [4]:
x_train, y_train = torch.load('trainX_SOLUSDT.pt', weights_only=True).to(torch.float32), torch.load('trainY_SOLUSDT.pt', weights_only=True).to(torch.float32)
mean, std = x_train.squeeze().reshape(-1).mean().item(), x_train.squeeze().reshape(-1).std().item()
dataset = TensorDataset(x_train, y_train)
batch_size = 5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
gru = GRUNetwork(1, 30, 2, 10, components=1, mean=mean, scale=std)

criterion = nn.MSELoss()
optimizer = optim.Adam(gru.parameters(), lr=0.9)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

# Обучение модели
num_epochs = 50

for epoch in range(num_epochs):
    for i, (batch_x, batch_y) in enumerate(dataloader):
        gru.train()
        outputs = gru(batch_x)
        print(outputs, batch_y)
        optimizer.zero_grad()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        print(f"step {i}, loss {loss.item():.4f}")
    scheduler.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

KeyboardInterrupt: 